<a href="https://colab.research.google.com/github/joseffaghihi/PACE-PEACE-dicrete-and-Continious/blob/main/Poly2_Peace_(Synthetic).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import mean_squared_error
from scipy.integrate import tplquad, dblquad, quad
import time
from scipy.optimize import minimize

In [2]:
n = 1000000                                                    # The number of samples
mean_real_0, mean_real_1, mean_real_2 = 0, 0.5, -0.3           # Specifying the means of data
mu_real_0, mu_real_1, mu_real_2 = 1, 1.2, 1.5                  # Specifying the variances of data
coeff_real = [0.5,-1,-3,4,-2,-1,0,1.05,-1.5,2]                 # Specifying the coefficients of the polynomial of degeee 2 whose variables are x0, x1 and x2. The order of the terms in this polynomial is as follows: 1, x0, x1, x2, x0^2, x0x1, x0x2, x1^2, x1x2, x2^2.
np.random.seed(0)
x0 = np.random.normal(mean_real_0, mu_real_0, n).flatten()     # Generating x0 from a normal distribution with the above mean and variance. 
x1 = np.random.normal(mean_real_1, mu_real_1, n).flatten()     # Generating x1 from a normal distribution with the above mean and variance.
x2 = np.random.normal(mean_real_2, mu_real_2, n).flatten()     # Generating x2 from a normal distribution with the above mean and variance.
                  
# Generating y from the polynomial with the above coefficients. Indeed, a random error is applied on the coefficients. 
y = (0.01*np.random.rand(1)[0]+coeff_real[0]) + (0.01*np.random.rand(1)[0]+coeff_real[1])*x0 +(0.01*np.random.rand(1)[0]+coeff_real[2])*x1 + (0.01*np.random.rand(1)[0]+coeff_real[3])*x2 + (0.01*np.random.rand(1)[0]+coeff_real[4])*x0**2 + (0.01*np.random.rand(1)[0]+coeff_real[5])*x0*x1 + (0.01*np.random.rand(1)[0]+coeff_real[6])*x0*x2 + (0.01*np.random.rand(1)[0]+coeff_real[7])*x1**2 + (0.01*np.random.rand(1)[0]+coeff_real[8])*x1*x2 + (0.01*np.random.rand(1)[0]+coeff_real[9])*x2**2

# Defining a dataframe from the above data.
data = np.zeros((n,4))
data[:,0], data[:,1], data[:,2], data[:,3] = [x0,x1,x2, y]
df = pd.DataFrame(data)
cols = ["x0", "x1", "x2", "y"]
df.columns = cols
df.head()

,x0,x1,x2,y
0,1.764052,1.117096,-0.650367,-12.117911
1,0.400157,1.835437,1.149102,1.203948
2,0.978738,1.310029,0.366006,-4.751005
3,2.240893,-0.172668,2.205574,8.366291
4,1.867558,-0.501878,0.744891,-0.939729


In [3]:
# Renaming the columns x0, x1, x2 to 0,1 and 2, respectively.

df = df.rename(columns={'x0':0, 'x1':1, 'x2':2})
df.head()

,0,1,2,y
0,1.764052,1.117096,-0.650367,-12.117911
1,0.400157,1.835437,1.149102,1.203948
2,0.978738,1.310029,0.366006,-4.751005
3,2.240893,-0.172668,2.205574,8.366291
4,1.867558,-0.501878,0.744891,-0.939729


In [4]:
# Training a polynomial regression for data.

X = np.array(df[[0,1,2]]).reshape(-1, 3)
Y = np.array(df['y']).reshape(-1, 1)
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.25)
poly_2 = PolynomialFeatures(degree = 2)
X_train_poly_2 = poly_2.fit_transform(X_train)
poly_2.fit(X_train_poly_2, Y_train)
lin2 = LinearRegression()
lin2.fit(X_train_poly_2, Y_train)
coeff = lin2.coef_[0]

In [5]:
# Evaluating the model using "r2_score" and "mean_squared_error"

expected_Y  = Y_test
predicted_poly_2_Y = lin2.predict(poly_2.fit_transform(X_test))
print(metrics.r2_score(expected_Y, predicted_poly_2_Y))
print(mean_squared_error(expected_Y, predicted_poly_2_Y))

1.0
9.785666667147978e-27


In [6]:
# The order of terms in the above polynomial model (i.e., [a,b,c] means (x0^a)(x1^b)(x2^c))

poly_2.powers_

array([[0, 0, 0],
       [1, 0, 0],
       [0, 1, 0],
       [0, 0, 1],
       [2, 0, 0],
       [1, 1, 0],
       [1, 0, 1],
       [0, 2, 0],
       [0, 1, 1],
       [0, 0, 2]])

In [7]:
# Calculating the means and the standard deviations of each column of the data.

df_ar = df.to_numpy()
mean = np.mean(df_ar.transpose()[0]), np.mean(df_ar.transpose()[1]) , np.mean(df_ar.transpose()[2])
sd = np.sqrt(float(np.cov(df_ar.transpose()[0]))), np.sqrt(float(np.cov(df_ar.transpose()[1]))), np.sqrt(float(np.cov(df_ar.transpose()[2])))



# Defining the density functions of each column of the data. Indeed, density(i, ) is the density function associated to the column i.

def density(i,x):
  prob_density = (1/(np.sqrt(np.pi*2)*sd[i])) * np.exp(-0.5*((x-mean[i])/sd[i])**2)
  return prob_density

In [9]:
# Calculating the PEACE's of x0, x1 and x2 on y.
tic = time.time()
inf = 8
integrand_0 = lambda x0,x1,x2: density(0,x0)*density(0,x0)*density(1,x1)*density(2,x2)*abs(coeff[1] + 2*coeff[4]*x0 + coeff[5]*x1 + coeff[6]*x2)
integrand_1 = lambda x0,x1,x2: density(0,x0)*density(1,x1)*density(1,x1)*density(2,x2)*abs(coeff[2] + coeff[5]*x0 + 2*coeff[7]*x1 + coeff[8]*x2)
integrand_2 = lambda x0,x1,x2: density(0,x0)*density(1,x1)*density(2,x2)*density(2,x2)*abs(coeff[3] + coeff[6]*x0 + coeff[8]*x1 + 2*coeff[9]*x2)
x01,x02 = -inf, inf
x11,x12 = lambda x0: -inf, lambda x0: inf
x21,x22 = lambda x0,x1: -inf, lambda x0,x1: inf
integral_0 = tplquad(integrand_0, x01, x02, x11, x12, x21, x22 )[0]
integral_1 = tplquad(integrand_1, x01, x02, x11, x12, x21, x22 )[0]
integral_2 = tplquad(integrand_2, x01, x02, x11, x12, x21, x22 )[0]
print("The PEACE's of x0,x1 and x2 on y are as follows, respectively:")
print(4*integral_0, 4*integral_1, 4*integral_2, sep = ',')
toc = time.time()
print("The runtime is ", toc - tic, "seconds.")

The PEACE's of x0,x1 and x2 on y are as follows, respectively:
3.080983185108964,2.5478350721580414,3.037782412700528
The runtime is  161.3133990764618 seconds.


In [10]:
pip install vegas

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.3 MB 27.1 MB/s 
     |████████████████████████████████| 1.9 MB 45.6 MB/s 
  Created wheel for vegas: filename=vegas-5.1.1-cp37-cp37m-linux_x86_64.whl size=1501838 sha256=8604249864104979a3b1d54d23a7e203acb4eac496ed9b92428cfa0f518eed25
  Stored in directory: /root/.cache/pip/wheels/5c/e4/05/3bcde90bc37d2c2b77a85b376b36926daee5516139fa9cd610
  Created wheel for gvar: filename=gvar-11.10.1-cp37-cp37m-linux_x86_64.whl size=4090136 sha256=1d3b1ce7db6e5a5fa0ee4432c1e14fa39fbb4c10b4cac1f3a69c479e070cdb87
  Stored in directory: /root/.cache/pip/wheels/ac/bd/b5/2b732c368e56f2d6151ff6a2fb3f3b118725516853fedf6435
Successfully built vegas gvar


In [11]:
import vegas

In [15]:
# Calculating the above PEACE's using Vegas

tic = time.time()
def integrand_0(X):
  return density(0,X[0])*density(0,X[0])*density(1,X[1])*density(2,X[2])*abs(coeff[1] + 2*coeff[4]*X[0] + coeff[5]*X[1] + coeff[6]*X[2])
def integrand_1(X):
  return density(0,X[0])*density(1,X[1])*density(1,X[1])*density(2,X[2])*abs(coeff[2] + coeff[5]*X[0] + 2*coeff[7]*X[1] + coeff[8]*X[2])
def integrand_2(X):
  return density(0,X[0])*density(1,X[1])*density(2,X[2])*density(2,X[2])*abs(coeff[3] + coeff[6]*X[0] + coeff[8]*X[1] + 2*coeff[9]*X[2])
  
domain = vegas.Integrator([[-inf, inf], [-inf, inf], [-inf, inf]])

integ_0, integ_1, integ_2 = domain(integrand_0, nitn=50, neval=10000), domain(integrand_1, nitn=50, neval=10000), domain(integrand_2, nitn=50, neval=10000)
print("The PEACE's of x0,x1 and x2 on y are as follows, respectively:")
print(4*integ_0, 4*integ_1, 4*integ_2,'\n', sep = '\n')
toc = time.time()
print("The runtime is ", toc - tic, "seconds.")

The PEACE's of x0,x1 and x2 on y are as follows, respectively:
3.08150(76)
2.54692(65)
3.03795(73)


The runtime is  28.51358652114868 seconds.


In [25]:
# Calculating the positive PEACE's using Vegas

tic = time.time()
def integrand_0(X):
  return density(0,X[0])*density(0,X[0])*density(1,X[1])*density(2,X[2])*max(coeff[1] + 2*coeff[4]*X[0] + coeff[5]*X[1] + coeff[6]*X[2], 0)
def integrand_1(X):
  return density(0,X[0])*density(1,X[1])*density(1,X[1])*density(2,X[2])*max(coeff[2] + coeff[5]*X[0] + 2*coeff[7]*X[1] + coeff[8]*X[2], 0)
def integrand_2(X):
  return density(0,X[0])*density(1,X[1])*density(2,X[2])*density(2,X[2])*max(coeff[3] + coeff[6]*X[0] + coeff[8]*X[1] + 2*coeff[9]*X[2], 0)
  
domain = vegas.Integrator([[-inf, inf], [-inf, inf], [-inf, inf]])

integ_0, integ_1, integ_2 = domain(integrand_0, nitn=50, neval=10000), domain(integrand_1, nitn=50, neval=10000), domain(integrand_2, nitn=50, neval=10000)
print("The positive PEACE's of x0,x1 and x2 on y are as follows, respectively:")
print(4*integ_0, 4*integ_1, 4*integ_2, sep = ',')
toc = time.time()
print("The runtime is ", toc - tic, "seconds.")

The positive PEACE's of x0,x1 and x2 on y are as follows, respectively:
0.69568(15),0.56990(17),2.28813(51)
The runtime is  25.447641611099243 seconds.


In [26]:
# Calculating the negative PEACE's using Vegas

tic = time.time()
def integrand_0(X):
  return density(0,X[0])*density(0,X[0])*density(1,X[1])*density(2,X[2])*max(-(coeff[1] + 2*coeff[4]*X[0] + coeff[5]*X[1] + coeff[6]*X[2]), 0)
def integrand_1(X):
  return density(0,X[0])*density(1,X[1])*density(1,X[1])*density(2,X[2])*max(-(coeff[2] + coeff[5]*X[0] + 2*coeff[7]*X[1] + coeff[8]*X[2]), 0)
def integrand_2(X):
  return density(0,X[0])*density(1,X[1])*density(2,X[2])*density(2,X[2])*max(-(coeff[3] + coeff[6]*X[0] + coeff[8]*X[1] + 2*coeff[9]*X[2]), 0)
  
domain = vegas.Integrator([[-inf, inf], [-inf, inf], [-inf, inf]])

integ_0, integ_1, integ_2 = domain(integrand_0, nitn=50, neval=10000), domain(integrand_1, nitn=50, neval=10000), domain(integrand_2, nitn=50, neval=10000)
print("The negative PEACE's of x0,x1 and x2 on y are as follows, respectively:")
print(4*integ_0, 4*integ_1, 4*integ_2, sep = ',')
toc = time.time()
print("The runtime is ", toc - tic, "seconds.")

The negative PEACE's of x0,x1 and x2 on y are as follows, respectively:
2.38353(48),1.97771(43),0.74805(18)
The runtime is  25.23659062385559 seconds.


In [27]:
# Defining a function which predicts y for a given (x0,x1,x2) using the above polynomial model.

def g(x0,x1,x2):
  return lin2.predict(poly_2.fit_transform(np.array([[x0,x1,x2]])))[0][0]

In [32]:
# Checking if PACE = PEACE

def opt_0_1(x):
  def f(z):
    return abs(coeff[1] + 2*coeff[4]*z + coeff[5]*x[0] + coeff[6]*x[1])*(density(0,z)**2)
  func = lambda a: (g(a[1],x[0],x[1])- g(a[0],x[0],x[1]))*density(0,a[0])*density(0,a[1])- quad(f,a[0],a[1])[0] 
  cons = ({'type': 'ineq', 'fun': lambda a:  g(a[1],x[0],x[1])- g(a[0],x[0],x[1])}, {'type': 'ineq', 'fun': lambda a:  a[1]- a[0]})
  a0 = ([1,1])
  bounds = [(None, None), (None, None)]
  return minimize(func, a0, bounds=bounds,constraints=cons).fun

def opt_0_2(x):
  def f(z):
    return abs(coeff[1] + 2*coeff[4]*z + coeff[5]*x[0] + coeff[6]*x[1])*(density(0,z)**2)
  func = lambda a: (g(a[0],x[0],x[1])- g(a[1],x[0],x[1]))*density(0,a[0])*density(0,a[1])- quad(f,a[0],a[1])[0] 
  cons = ({'type': 'ineq', 'fun': lambda a:  g(a[0],x[0],x[1])- g(a[1],x[0],x[1])}, {'type': 'ineq', 'fun': lambda a:  a[1]- a[0]})
  a0 = ([1,1])
  bounds = [(None, None), (None, None)]
  return minimize(func, a0, bounds=bounds,constraints=cons).fun

def opt_1_1(x):
  def f(z):
    return abs(coeff[2] + coeff[5]*x[0] + 2*coeff[7]*z + coeff[8]*x[1])*(density(1,z)**2)
  func = lambda a: (g(x[0],a[1],x[1])- g(x[0],a[0],x[1]))*density(1,a[0])*density(1,a[1])- quad(f,a[0],a[1])[0] 
  cons = ({'type': 'ineq', 'fun': lambda a:  g(x[0],a[1],x[1])- g(x[0],a[0],x[1])}, {'type': 'ineq', 'fun': lambda a:  a[1]- a[0]})
  a0 = ([1,1])
  bounds = [(None, None), (None, None)]
  return minimize(func, a0, bounds=bounds,constraints=cons).fun

def opt_1_2(x):
  def f(z):
    return abs(coeff[2] + coeff[5]*x[0] + 2*coeff[7]*z + coeff[8]*x[1])*(density(1,z)**2)
  func = lambda a: (g(x[0],a[0],x[1])- g(x[0],a[1],x[1]))*density(1,a[0])*density(1,a[1])- quad(f,a[0],a[1])[0] 
  cons = ({'type': 'ineq', 'fun': lambda a:  g(x[0],a[0],x[1])- g(x[0],a[1],x[1])}, {'type': 'ineq', 'fun': lambda a:  a[1]- a[0]})
  a0 = ([1,1])
  bounds = [(None, None), (None, None)]
  return minimize(func, a0, bounds=bounds,constraints=cons).fun

def opt_2_1(x):
  def f(z):
    return abs(coeff[3] + coeff[6]*x[0] + coeff[8]*x[1] + 2*coeff[9]*z)*(density(1,z)**2)
  func = lambda a: (g(x[0],x[1],a[1])- g(x[0],x[1],a[0]))*density(2,a[0])*density(2,a[1])- quad(f,a[0],a[1])[0] 
  cons = ({'type': 'ineq', 'fun': lambda a:  g(x[0],x[1],a[1])- g(x[0],x[1],a[0])}, {'type': 'ineq', 'fun': lambda a:  a[1]- a[0]})
  a0 = ([1,1])
  bounds = [(None, None), (None, None)]
  return minimize(func, a0, bounds=bounds,constraints=cons).fun

def opt_2_2(x):
  def f(z):
    return abs(coeff[3] + coeff[6]*x[0] + coeff[8]*x[1] + 2*coeff[9]*z)*(density(1,z)**2)
  func = lambda a: (g(x[0],x[1],a[0])- g(x[0],x[1],a[1]))*density(2,a[0])*density(2,a[1])- quad(f,a[0],a[1])[0]  
  cons = ({'type': 'ineq', 'fun': lambda a:  g(x[0],x[1],a[0])- g(x[0],x[1],a[1])}, {'type': 'ineq', 'fun': lambda a:  a[1]- a[0]})
  a0 = ([1,1])
  bounds = [(None, None), (None, None)]
  return minimize(func, a0, bounds=bounds,constraints=cons).fun

func_0_1 = lambda x: opt_0_1(x)
func_0_2 = lambda x: opt_0_2(x)
func_1_1 = lambda x: opt_1_1(x)
func_1_2 = lambda x: opt_1_2(x)
func_2_1 = lambda x: opt_2_1(x)
func_2_2 = lambda x: opt_2_2(x)

a0 = ([1,1])
bounds = [(None, None), (None, None)]
minimize(func_0_1, a0, bounds=bounds).fun, minimize(func_0_2, a0, bounds=bounds).fun, minimize(func_1_1, a0, bounds=bounds).fun, minimize(func_1_2, a0, bounds=bounds).fun, minimize(func_2_1, a0, bounds=bounds).fun, minimize(func_2_2, a0, bounds=bounds).fun

(0.0, 0.0, 0.0, 0.0, -1.1317396925004812, 0.0)